In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/landimark_project/health_result.csv')
df.head()

,question,answer,category02
0,"너무 목이 마른데요, 혹시 정수기 같은 건 없나요?",정수기는 진료 대기실 앞 쪽에 있습니다. 감사합니다.,29.대기실 및 진료실 위치 안내
1,교정기가 떨어진 것 같아서 붙이러 왔어요.,접수 먼저 부탁드립니다. 카운터에서 성함을 입력해 주시길 바랍니다.,27.진료 접수 안내
2,앞으로 얼마나 나아지는지 보고 아프면 오라고 하네요. 제가 다음에 예약하고 와도 되죠?,예약하실 때 지난 번 진료 기록이 있다고 말씀해 주시길 부탁 드립니다. 감사합니다.,33.다음 진료실 예약
3,이것도 수납용 키오스크인가요? 대기 줄이 너무 길어서요.,이곳에서도 수납 진행 가능합니다. 홈버튼을 누른 후 수납 버튼을 누르십시오.,31.수납 방법 안내
4,주사실 안에 가서 있으라는데 주사실이 어딘데요?,침을 맞는 곳이라 이름이 주사실입니다. 3층으로 올라가시면 됩니다.,29.대기실 및 진료실 위치 안내


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# KcBERT 모델 로드 (한국어 모욕성 표현 감지)
tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcBERT-base")

def detect_offensive(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs)
    prediction = outputs.logits.argmax(dim=-1)
    return prediction.item()  # 0: 정상 문장, 1: 모욕적 문장

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 필요한 열 선택 (예: 'question', 'answer')
questions = df['question'].tolist()
answers = df['answer'].tolist()

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
# Tokenizer 및 모델 로드 (예: BERT)
tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcBERT-base", num_labels=len(set(answers)))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 질문을 토크나이즈
encodings = tokenizer(questions, truncation=True, padding=True, return_tensors="pt")

# 대답을 레이블로 변환
labels = torch.tensor([answers.index(a) for a in answers])

In [ ]:
# 레이블을 생성하는 부분에서 문제가 없는지 확인
label_set = set(answers)  # 고유 레이블 세트
num_labels = len(label_set)  # 레이블 개수

print("레이블 개수:", num_labels)

# 레이블 인덱스 매핑
label_to_idx = {label: idx for idx, label in enumerate(sorted(label_set))}
labels = torch.tensor([label_to_idx[a] for a in answers])

# 레이블 값 확인
print("레이블 값:", labels)

레이블 개수: 8919
레이블 값: tensor([6512, 6381, 5082,  ..., 1558, 8883, 7535])


In [ ]:
print(labels.min(), labels.max())  # 최소값과 최대값 확인

tensor(0) tensor(8918)


In [ ]:
# Dataset 클래스 정의
class QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Dataset 객체 생성
dataset = QADataset(encodings, labels)

In [ ]:
# 학습 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

In [ ]:
# 모델 학습
trainer.train()

Step,Training Loss
10,9.201900


In [ ]:
# 모델 성능 평가
eval_results = trainer.evaluate()
print(eval_results)

NameError: name 'trainer' is not defined